In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_columns = 50
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import sklearn

## Load in `.csv` file

In [9]:
df = pd.read_csv('data/extracted/crime-data-from-2010-to-present.csv')

In [ ]:
df.head()

In [ ]:
df.shape

## Converting `date occured` to time series index

In [10]:
# removing empty time stamp from Date Reported and Date Ocurred
df["Date Reported"] = df["Date Reported"].str.replace('T00:00:00', '')
df["Date Occurred"] = df["Date Occurred"].str.replace('T00:00:00', '')

In [ ]:
# setting 'date occured' column as datetime object
df['Date Occurred'] = pd.to_datetime(df['Date Occurred'], format= '%Y-%m-%d')

In [ ]:
df = df.set_index('Date Occurred').sort_index()

In [ ]:
# checking that it all worked
df.head(3)

## Data Cleaning
Now that we have the dataframe converted to a format for time series analysis, we can do basic data cleaning steps.

In [ ]:
# checking for null values
df.isna().sum()

In [ ]:
# dropping columns with a majority of missing values
df = df.drop(axis=1, columns=['MO Codes', 
                              'Weapon Used Code', 
                              'Weapon Description', 
                              'Crime Code 1',
                              'Crime Code 2', 
                              'Crime Code 3', 
                              'Crime Code 4', 
                              'Cross Street'])

In [ ]:
demo_features = ['Victim Age', 'Victim Sex', 'Victim Descent']

In [ ]:
df.isna().sum()

In [ ]:
# now we just have to deal with the remaining null values...

In [ ]:
df.info()

In [ ]:
# df['Date Reported'] = pd.to_datetime(df['Date Reported'], format= '%Y-%m-%d')
df.loc[df['Date Reported'] == '1022019-01-01']

In [ ]:
# checking for all values that are ***2019-01-01
unknown = []
for i in range(len(df['Date Reported'])): 
    if len(df['Date Reported'][i])==13:
        unknown.append(df['Date Reported'][i])
len(unknown)


In [ ]:
#replacing all ***2019-01-01 with 2019-01-01
for i in range(len(df['Date Reported'])): 
    if len(df['Date Reported'][i])==13:
        df['Date Reported'] = '2019-01-01'

In [ ]:
#converting date reported variable to datetime format
df['Date Reported'] = pd.to_datetime(df['Date Reported'], format= '%Y-%m-%d')

In [ ]:
df['Premise Code'].fillna(0, inplace = True)
df['Premise Description'].fillna('UNKNOWN', inplace = True)
df['Status Code'].fillna('UNK', inplace = True)

# David

In [ ]:
df.isna().sum()

In [ ]:
# let's change H, N, and - to X for Unknown/Other
df['Victim Sex'].value_counts()

In [ ]:
# need to strip whitespace from column name and break out lat/lon data
df['Location '][0]

In [ ]:
df.head(2)

In [ ]:
df['data_points'] = df.index.value_counts()

In [ ]:
crime_series = df['data_points']

In [ ]:
crime_series.dtype

In [ ]:
def plot_series(series):
    plt.figure(figsize=(12,6))
    plt.plot(crime_series)
    plt.ylabel('Crimes per day in LA', fontsize=15)
    
    for year in range(2010,2021):
        plt.axvline(datetime(year,1,1), linestyle='--', alpha=0.5)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(crime_series);

In [ ]:
# plot_series(crime_series)

# Normalize Data

In [ ]:
mean, std = crime_series.mean(), crime_series.std()

In [ ]:
crime_series = (crime_series - mean) / std

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(crime_series);

# Differencing

In [ ]:
crime_series = crime_series.diff()

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(crime_series);

In [ ]:
crime_series.tail()

# Reduce Volatility

In [ ]:
annual_volatility = crime_series.groupby(crime_series.index.year).std()

In [ ]:
annual_volatility

# Reevaluate with Dickey Fuller Test

## EDA

Essentially the variables can be grouped to 
* When the crime occurred - given by date and time
* When the crime was reported - Date
* what was the crime - crime code and crime description
* Who was the victim - Victim sex, age and descent
* What weapon was involved - weapon code and description
* Where the crime took place - given by location, address, cross street, premise code and description

In [ ]:
df.data_points['2010-01-01':'2017-12-31']

In [ ]:
df.groupby('Date Occurred').sum()

# Data Visuals / EDA

In [ ]:
clean_df = pd.read_pickle('data/pickled/cleaned_df.pkl')

In [ ]:
clean_df.head(2)

In [11]:
### USING df FROM ABOVE, NOT clean_df

s = pd.to_datetime(df['Date Occurred'], format= '%Y-%m-%d')
df = s.groupby(s.dt.floor('d')).size().reset_index(name='count')
print (df)

     Date Occurred  count
0       2010-01-01   2222
1       2010-01-02    533
2       2010-01-03    539
3       2010-01-04    558
4       2010-01-05    547
...            ...    ...
3455    2019-06-18    497
3456    2019-06-19    534
3457    2019-06-20    459
3458    2019-06-21    505
3459    2019-06-22    498

[3460 rows x 2 columns]


In [ ]:
df.head()

In [ ]:
# clean_df['data_points'] = clean_df.index.value_counts()
# crime_series = clean_df['data_points']
# crime_series.dtype

In [12]:
mean, std = df.mean(), df.std()
print(mean)
print(std)

count    576.086416
dtype: float64
count    89.547556
dtype: float64


In [14]:
df['count'] = (df['count'] - mean) / std

In [15]:
df.head()

,Date Occurred,count
0,2010-01-01,NaN
1,2010-01-02,NaN
2,2010-01-03,NaN
3,2010-01-04,NaN
4,2010-01-05,NaN


In [ ]:
plt.figure(figsize=(12,6))
plt.plot(df);

In [ ]:
# Use pandas grouper to group values using annual frequency
year_groups = df.groupby(pd.Grouper(freq ='A'))

In [ ]:
# Create a new DataFrame and store yearly values in columns 
crime_annual = pd.DataFrame()

for yr, group in year_groups:
    crime_annual[yr.year] = group.values.ravel()
    
# Plot the yearly groups as subplots
crime_annual.plot(figsize = (13,8), subplots=True, legend=True);